In [ ]:
!pip install nglview
!pip install https://github.com/volkamerlab/opencadd/archive/master.tar.gz
!git clone https://github.com/maranasgroup/Protein-InDelMaker.git
# install nglview, which is a library to look at the proteins
# opencadd is a library that includes the code 编码？ of the enzymes
# protein-indelmaker is a bioinformatics tool used for identifying and analyzing insertions and deletions (indels) in protein sequences


In [ ]:
import sys
if 'google.colab' in sys.modules:
  !pip install pyrosettacolabsetup
  import pyrosettacolabsetup
  pyrosettacolabsetup.install_pyrosetta()
  exit()
# ensure pyrosetta is availbale to be downloaded
# pyrosetta is a Python library that provides a powerful framework for protein structure prediction, design, and analysis

import sys
if 'google.colab' in sys.modules:
  !pip install pyrosettacolabsetup
  import pyrosettacolabsetup
  pyrosettacolabsetup.mount_pyrosetta_install()
# importing PyRosetta
from pyrosetta import *
init()
# initiate the pyrosetta program

if not os.getenv("DEBUG"):
    !pip install attrs billiard biopython blosc dask dask-jobqueue distributed GitPython graphviz jupyter matplotlib numpy pandas py3Dmol scipy seaborn traitlets --user
# debug the program

In [ ]:
#%cd ../
!pip install -q condacolab
import condacolab
condacolab.install()
# import condacolab

#@title
!pip install condacolab
# install condacolab

import condacolab
condacolab.check()
# make sure condacolab is completely installed

In [ ]:
!conda install -c conda-forge -c bioconda mgltools openbabel mdanalysis redo numpy boost-cpp swig vina --yes
# install vina for protein docking

In [4]:
import sys
import warnings
import os
import shutil
import subprocess
from pathlib import Path
#import nglview as nv
from vina import Vina
from openbabel import pybel
from opencadd.structure.core import Structure


# filter warnings to allow the program to run
warnings.filterwarnings("ignore")
ob_log_handler = pybel.ob.OBMessageHandler()
pybel.ob.obErrorLog.SetOutputLevel(0)

# define paths to make directories for existing libraries
def makeDatadirc(path):
    if not os.path.exists(path):
        os.makedirs(path)
    else: pass
        #shutil.rmtree(path)  # Removes all the subdirectories!
        #os.makedirs(path)


HERE = os.getcwd()
DATA = HERE + "/data"
print(DATA)
#to check the directory and know where the current directory is

makeDatadirc(DATA)
makeDatadirc('AutodockUtilities24')
# make a data directory for uploading existing libraries

# retrieve structure from the Protein Data Bank
pdb_id = "2qzs"
structure = Structure.from_pdbid(pdb_id)
# element information maybe missing, but important for subsequent PDBQT conversion
if not hasattr(structure.atoms, "elements"):
    structure.add_TopologyAttr("elements", structure.atoms.types)
structure
# NBVAL_CHECK_OUTPUT


# write the protein file to disk
protein = structure.select_atoms("protein")
protein.write(DATA +"/protein.pdb")

/Users/peng/work/vina/data


In [26]:
import re
str = './data/2qzs.18_V/2qzs.18_V.pdbqt'

# re.search(r'/\w+\.', str[:-6]).group(0)[1:-1]

str.split('.')[-2]

'18_V'

In [6]:
def pdb_to_pdbqt(pdb_path, pdbqt_path):
    """
    Convert a PDB file to a PDBQT file needed by docking programs of the AutoDock family.
    Parameters
    ----------
    pdb_path: str or pathlib.Path
        Path to input PDB file.
    pdbqt_path: str or pathlib.path
        Path to output PDBQT file.
    """
    print(os.getcwd())
    # print(os.chdir('./data'))
    # print(pdb_path)
    # print(os.getcwd())
    
    _cmd = ['python',
            './AutodockUtilities24/prepare_receptor4.py',
            '-v',
            '-r',
            pdb_path,
            '-o', 
            pdbqt_path,
            '-A',
            'checkhydrogens',
            '-U',
            'waters']
    result = subprocess.run(_cmd, capture_output=True, text=True)
    print(result.stdout)
    print(result.stderr)
    resulttext = [result.stdout,result.stderr]
    return resulttext

# convert protein to PDBQT format
pdb_to_pdbqt("result_2.pdb", "result_2.pdbqt")

/Users/peng/work/vina
set verbose to  True
set receptor_filename to  result_2.pdb
set outputfilename to  result_2.pdbqt
set repairs to  checkhydrogens
set cleanup to  waters

Traceback (most recent call last):
  File "/Users/peng/work/vina/./AutodockUtilities24/prepare_receptor4.py", line 157, in <module>
    mols = Read(receptor_filename)
  File "/Users/peng/opt/anaconda3/envs/vina/lib/python3.9/site-packages/MolKit/__init__.py", line 38, in Read
    raise AssertionError("%s does't exist" % filename)
AssertionError: result_2.pdb does't exist



['set verbose to  True\nset receptor_filename to  result_2.pdb\nset outputfilename to  result_2.pdbqt\nset repairs to  checkhydrogens\nset cleanup to  waters\n',
 'Traceback (most recent call last):\n  File "/Users/peng/work/vina/./AutodockUtilities24/prepare_receptor4.py", line 157, in <module>\n    mols = Read(receptor_filename)\n  File "/Users/peng/opt/anaconda3/envs/vina/lib/python3.9/site-packages/MolKit/__init__.py", line 38, in Read\n    raise AssertionError("%s does\'t exist" % filename)\nAssertionError: result_2.pdb does\'t exist\n']

If you have already got these pdbqt files, you can ignore this cell.

In [ ]:
# define ligand sdf for protein-ligand complex of interest
sdf = 'ADP-glucose.sdf'
sdf2 ='Maltotriose.sdf'
sdf3 ='Amylose.sdf'
def sdf_to_pdbqt(sdf, pdbqt_path, pH=7.4):
    """
    Convert a sdf string to a PDBQT file needed by docking programs of the AutoDock family.
    Parameters
    ----------
    sdf: file
        sdf file.
    pdbqt_path: str or pathlib.path
        Path to output PDBQT file.
    pH: float
        Protonation at given pH.
    """
    molecule = pybel.readfile("sdf", sdf).__next__()
    # add hydrogens at given pH
    molecule.OBMol.CorrectForPH(pH)
    molecule.addh()
    # generate 3D coordinates
    molecule.make3D(forcefield="mmff94s", steps=10000)
    # add partial charges to each atom
    for atom in molecule.atoms:
        atom.OBAtom.GetPartialCharge()
    molecule.write("pdbqt", str(pdbqt_path), overwrite=True,)
    return
# convert the ligand into PDBQT format
os.chdir('/content/data')

sdf_to_pdbqt(sdf, DATA+"/ADPG.pdbqt")
sdf_to_pdbqt(sdf2, DATA+"/Maltotriose.pdbqt")
sdf_to_pdbqt(sdf3, DATA+"/Amylose.pdbqt")

In [2]:
from psutil import *
print(cpu_count(), cpu_stats())


24 scpustats(ctx_switches=3693, interrupts=2113576, soft_interrupts=4294967295, syscalls=2958701)


In [ ]:
%cd /content/
!chmod u+x ./vina_1.2.3_linux_x86_64
!./vina_1.2.3_linux_x86_64 --help

/content
AutoDock Vina v1.2.3

Input:
  --receptor arg             rigid part of the receptor (PDBQT)
  --flex arg                 flexible side chains, if any (PDBQT)
  --ligand arg               ligand (PDBQT)
  --batch arg                batch ligand (PDBQT)
  --scoring arg (=vina)      scoring function (ad4, vina or vinardo)

Search space (required):
  --maps arg                 affinity maps for the autodock4.2 (ad4) or vina 
                             scoring function
  --center_x arg             X coordinate of the center (Angstrom)
  --center_y arg             Y coordinate of the center (Angstrom)
  --center_z arg             Z coordinate of the center (Angstrom)
  --size_x arg               size in the X dimension (Angstrom)
  --size_y arg               size in the Y dimension (Angstrom)
  --size_z arg               size in the Z dimension (Angstrom)
  --autobox                  set maps dimensions based on input ligand(s) (for 
                             --score_only and 

In [ ]:
ligand_resname = "ADP"
ligand = structure.select_atoms(f"resname {ligand_resname}")

pocket_center = (ligand.positions.max(axis=0) + ligand.positions.min(axis=0))/2
pocket_size = ligand.positions.max(axis=0) - ligand.positions.min(axis=0) + 20
print(pocket_center)
print(float(pocket_center[0]))
print(pocket_size)
print(float(pocket_size[0]), pocket_size[1], pocket_size[0])

[25.3575    -3.9050002 42.039    ]
25.357500076293945
[29.373001 28.172    27.708   ]
29.373001098632812 28.172 29.373001


In [ ]:
def print_vina_infor(
    Ligands, protein_path, out_path, pocket_center, pocket_size, num_poses=4, exhaustiveness=6, flex_protein=None
):

    print(os.chdir(
        '/content/'))
    print(os.getcwd())
    # config =subprocess.run('/Users/qixie/Library/CloudStorage/OneDrive-JohnsHopkins/JHU/Friedlab/pythonProject/DockingProject/AutodockUtilities24/prepare_configFile.py',
    #                '-a',str(pocket_center[0]),
    #                '-b',str(pocket_center[1]),
    #                '-c',str(pocket_center[2]),
    #                '-X',str(pocket_size[0]),
    #                '-Y',str(pocket_size[1]),
    #                'Z',str(pocket_size[2]),
    #                capture_output=True, text=True
    #                )
    # print(config.stdout)
    # print(config.stderr)
    print('./vina_1.2.3_linux_x86_64 '+
                             '--receptor '+ protein_path+

                             ' --ligand '+ str(Ligands[0]) +' '+str(Ligands[1])+
                             ' --center_x '+ str(pocket_center[0])+
                             ' --center_y '+ str(pocket_center[1])+
                             ' --center_z '+ str(pocket_center[2])+
                             ' --size_x '+ str(pocket_size[0])+
                             ' --size_y '+ str(pocket_size[1])+
                             ' --size_z '+ str(pocket_size[2])+
                             ' --out '+ str(out_path)+
                             ' --verbosity '+'2'+ ' --cpu ', '2',
                             ' --num_modes '+str(num_poses)+
                             ' --exhaustiveness '+str(exhaustiveness)

                            )

    #
    # v.set_receptor(protein_path)
    # v.set_ligand_from_file(str(ligand_path))
    # v.compute_vina_maps(center=[float(pocket_center[0]),
    #                             float(pocket_center[1]),
    #                             float(pocket_center[2])],
    #                     box_size=[float(pocket_size[0]),
    #                               float(pocket_size[1]),
    #                               float(pocket_size[0])
    #                               ])
    # energy = v.score()
    # print('Score before minimization: %.3f (kcal/mol)' % energy[0])
    # energy_minimized = v.optimize()
    # print('Score after minimization : %.3f (kcal/mol)' % energy_minimized[0])
    # v.write_pose(str(out_path), overwrite=True)
    #
    # v.dock(exhaustiveness=exhaustiveness, n_poses=num_poses)
    # output_text = v.write_poses(str(out_path), n_poses=5, overwrite=True)


Ligands=[(DATA + "/ADPG.pdbqt"),(DATA + "/Maltotriose.pdbqt")]
print(Ligands)
output_text = print_vina_infor(Ligands=Ligands,
                            protein_path=(DATA + "/result_1.pdbqt"),

                            out_path=(DATA + "/docking_poses_1_0619.pdbqt"),
                            pocket_center=pocket_center,
                            pocket_size=pocket_size)


['/content/data/ADPG.pdbqt', '/content/data/Maltotriose.pdbqt']
None
/content
./vina_1.2.3_linux_x86_64 --receptor /content/data/result_1.pdbqt --ligand /content/data/ADPG.pdbqt /content/data/Maltotriose.pdbqt --center_x 25.3575 --center_y -3.9050002 --center_z 42.039 --size_x 29.373001 --size_y 28.172 --size_z 27.708 --out /content/data/docking_poses_1_0619.pdbqt --verbosity 2 --cpu  2  --num_modes 4 --exhaustiveness 6


In [ ]:
!chmod u+x ./vina_1.2.3_linux_x86_64
!./vina_1.2.3_linux_x86_64 --receptor /content/data/result_2.pdbqt --ligand /content/data/ADPG.pdbqt /content/data/Maltotriose.pdbqt --center_x 25.3575 --center_y -3.9050002 --center_z 42.039 --size_x 29.373001 --size_y 28.172 --size_z 27.708 --out /content/data/docking_poses_2_0619.pdbqt --verbosity 2 --cpu  2  --num_modes 4 --exhaustiveness 6

AutoDock Vina v1.2.3
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V

In [ ]:
def feed_indel(Mutations):
  print(Mutations)
  mutationfile=open("mutation.input", "w")
  print(len(Mutations))
  for i in range (len(Mutations)):
    print ("MUTATE CHAIN_A "+str(Mutations[i]+"\n"))
    mutationfile.write("MUTATE CHAIN_A "+str(Mutations[i]+"\n"))
  mutationfile.close()

Mutations=['15_R','16_N','18_A']
feed_indel(Mutations)

In [ ]:
from pyrosetta import *
init()
!python /content